## Give me the best deck with Champion X

In [1]:
import pandas as pd

In [71]:
df = pd.read_csv(r'./../data/raw/lor_ranked_data_all.csv') # read in the data

In [137]:
champion = 'Tryndamere'
date = '2022-07-01'
n = 100

In [138]:
def best_deck_by_champ(df, champion, date, n):
    df_date = df[df['game_start_time_utc'] > date]
    champ_list = df_date[df_date['archetype'].str.contains(champion)]
    archetypes = champ_list['deck_code'].value_counts() # find the unique archetypes being played
    archetypes = archetypes[archetypes > n] #only keep popular deck_codes
    popular_list = list(archetypes.index) # make a list of popular deck_codes
    champ_list = champ_list.loc[df['deck_code'].isin(popular_list)] # filter df to only keep popular archetypes
    df_champ = champ_list.groupby(['deck_code', 'game_outcome']).size()
    df_champ = df_champ.groupby(level=0).transform(lambda x: (x / x.sum()).round(2)*100)
    df_champ.drop('loss', level=1, axis=0, inplace=True) #drop loss %
    df_champ = df_champ.droplevel(level=1) #drop win/loss
    df_champ = df_champ.sort_values(ascending=False)
    
    df_small = df_date[['deck_code', 'archetype']]
    y = pd.DataFrame(df_champ)
    
    inner = pd.merge(y, df_small, on='deck_code', how='inner')#.value_counts()
    inner = inner.value_counts()
    df_champ = inner.sort_values(ascending=False)
    
    return df_champ

In [139]:
best_deck_by_champ(df, champion, date, n) # sorted by playrate

deck_code                                                                                 0     archetype                        
CECACBABBYBACBIBFABQCAIME4ZAGAYBAYFBMBABAEARIAIDAEBQCBABBIBACBIZDUBACAYBCMBACBIPCM        54.0  Tryndamere Trundle (SI)              416
CECACBABBYBACAIME4BACBIBFACAGAIDAYFBMBABAECR2AIDAEJQCBABBIBACAIUGQBACAIBAEBACBIPDE        53.0  Tryndamere Trundle (SI)              351
CECACBABBYBACAIME4BQCBIBDUUAIAYBAMDAUFQDAEAQKDYBAMARGAQBAEARIAQBAEATIAIBAUMQ              57.0  Tryndamere Trundle (SI)              319
CECACBABBYBACBIBFABQCAIME4ZAGAYBAYFBMAYBAMAQGAIEAEGAGAIBAEKDIAQBAMARGAQBAUMR2             0.0   Tryndamere Trundle (SI)              260
                                                                                          51.0  Tryndamere Trundle (SI)              260
CECACBABBYBACAIME4BACBIBFABQGAIGBILAEAIDAEBQGAIBAMKDEBABAMARGAIDAUGAEAIBAE2AIAIFB4JRSHI   54.0  Tryndamere Trundle (SI)              235
CECACAIBBQAQIAIOAIAQKAJIAMBQCAYGBICACAIFDUAQGAIW

In [140]:
def best_deck_by_champ_by_winrate():
    x = best_deck_by_champ(df, champion, date, n)
    df_x = pd.DataFrame(x)
    df_x.index.names = ["deck_code", "winrate", "archetype"]
    df_x.reset_index(level=['winrate', 'archetype'], inplace=True)
    df_x = df_x.sort_values(by=['winrate'], ascending=False)
    return df_x
best_deck_by_champ_by_winrate()

,winrate,archetype,0
deck_code,,,
CECACAIBAMAQIAIOAIBQCBQWAMAQKAI5FACACAYBAMAQIAIKAIAQKDYZAUAQCAIMCQTTEAIBAECSC,66.0,Tryndamere Trundle (SI),109
CECACBABBYBACAIME4BACBIBFABQGAIGBILAIAIBAUOQCAYBAMAQIAIKAMAQCFBSGQBACAYBCMBQCBIPCMMQ,62.0,Tryndamere Trundle (SI),110
CECACBABBYBACAIME4BACBIBFACAGAIDAYFBMBABAECR2AIDAEJQCBABBIBACAIUGQAQGAIFB4JRS,61.0,Tryndamere Trundle (SI),118
CECACBABBYBACAIME4BQCBIBDUUAIAYBAMDAUFQDAEAQKDYBAMARGAQBAEARIAQBAEATIAIBAUMQ,57.0,Tryndamere Trundle (SI),319
CECACBABBYBACBIBFABQCAIME4ZAGAYBAYFBMBABAEARIAIDAEBQCBABBIBACBIZDUBACAYBCMBACBIPCM,54.0,Tryndamere Trundle (SI),416
CECACBABBYBACAIME4BACBIBFABQGAIGBILAEAIDAEBQGAIBAMKDEBABAMARGAIDAUGAEAIBAE2AIAIFB4JRSHI,54.0,Tryndamere Trundle (SI),235
CIBQCAIBBQBAGAIKCYBQGCKAKZLQGAIEAEGAEAYBAMJQIAIBCQTSUNAEAEAQCFYBAMAQIAQEAECQ4BADBECRKF2U,54.0,Tryndamere Aurelion Sol Lissandra,147
CECACBABBYBACBIBFABQCAIME4ZAGAYBAYFBMBABAEARIAIBAUOQCAYBAMAQIAIKAMAQCAJUAEBQCEYDAECQ6EYZ,54.0,Tryndamere Trundle (SI),107
CECACBABBYBACAIME4BACBIBFACAGAIDAYFBMBABAECR2AIDAEJQCBABBIBACAIUGQBACAIBAEBACBIPDE,53.0,Tryndamere Trundle (SI),351


In [132]:
# # double check it worked
# df_date = df[df['game_start_time_utc'] > date] #build dataframe you build in function
# deck = y.index[0][0] # get deck code returned for deck #1 in the function
# true_value = df_date.deck_code.str.count(deck).sum() #see how many times that deck actually appears in dataframe
# output_value = y[0] # get how many times the function said that deck appears
# true_value == output_value # compare values

In [110]:
## write code to stalk someone

In [ ]:
contain_values = df[df['month'].str.contains('Ju')]
print (contain_values)

KeyError: 'month'

In [57]:
df_x = df.dropna

In [75]:
contain_values = df[df['player'].str.contains('J Tiller#NA1', na=False)]
contain_values['archetype'].value_counts()

Diana Aphelios (SI)    18
Trundle Gnar (PZ)       9
Lissandra Taliyah       6
Thresh Nasus            5
Fizz Riven (BC)         4
Ekko Sivir Zilean       3
Yuumi Pantheon (DE)     1
Elise Viego (NX)        1
Name: archetype, dtype: int64

In [96]:
contain_values = df[df['player'].str.contains('Majiin', na=False)]
contain_values['archetype'].value_counts()
# contain_values

Ahri Bard                  62
Ezreal Annie               56
Jarvan IV Poppy Bard       50
Twisted Fate Nami (SI)     45
Aphelios Viktor            11
Ezreal Kennen (IO)          9
Zoe Aphelios Yuumi (SI)     6
Elise Annie                 2
Tryndamere Trundle (IO)     2
Katarina Ahri               1
Name: archetype, dtype: int64

In [99]:
contain_values = df[df['player'].str.contains('Griffener#1217', na=False)]
contain_values['archetype'].value_counts()

Rumble Sion (BC)        4
Jarvan IV Illaoi        4
Jarvan IV Poppy Bard    2
Malphite Taliyah        1
Katarina Ahri Annie     1
Sivir Akshan (DE)       1
Ashe LeBlanc            1
Pyke Rek'Sai            1
Name: archetype, dtype: int64